In [28]:
import pickle
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split



df_list = []
# load processed_textblob_imm.pkl
with open('processed_textblob_imm.pkl', 'rb') as f:
    a= pickle.load(f)
    for sub in a:
        for i in sub:
            df_list.append(i)
            
print(len(df_list))

566


In [2]:
df_list[0].head()

,text,agu_1,imm,imm_check,txt_sentiment,imm_sentiment,txt_blob,imm_blob
id,,,,,,,,
cvs92ym,"&gt;So within our predefined two genders, we h...","{""summarization"": ""The speaker expresses frust...","""So, within our established binary gender fra...",True,"[0.04, 0.092, 0.868, -0.6548]","[0.088, 0.056, 0.856, 0.128]","[0.009090909090909115, 0.43787878787878787]","[-0.08, 0.735]"
cvstt70,&gt; But saying you're a boy because you act m...,"{""summarization"": ""The speaker expresses stron...","""But saying someone's male just because they ...",True,"[0.036, 0.046, 0.918, -0.2263]","[0.224, 0.126, 0.65, 0.6652]","[0.05, 0.18645833333333334]","[0.06825396825396826, 0.4626984126984126]"
cvsvr2g,"Because I have a penis, and balls, and my chro...",I cannot summarize the Reddit speaker's emotio...,"Because I have a penis, and balls, and my chro...",False,"[0.0, 0.0, 1.0, 0.0]","[0.0, 0.0, 1.0, 0.0]","[0.0, 0.1]","[0.0, 0.1]"
cvsvwuy,"&gt; Because I have a penis, and balls, and my...","{""summarization"": ""The speaker expresses frust...","&gt; Because I have a penis, and balls, and my...",False,"[0.027, 0.129, 0.844, -0.8451]","[0.027, 0.129, 0.844, -0.8451]","[-0.135, 0.44571428571428573]","[-0.135, 0.44571428571428573]"
cvsxaph,Like I said a million times before. Exceptions...,"{""summarization"": ""The speaker is likely frust...",Like I said a million times before. Exceptions...,False,"[0.048, 0.05, 0.902, -0.0433]","[0.048, 0.05, 0.902, -0.0433]","[-0.05113636363636364, 0.4231060606060606]","[-0.05113636363636364, 0.4231060606060606]"


In [30]:
# load the whole corpus 
from convokit import Corpus, Speaker, Utterance, Conversation, download
import os, pandas as pd

path = "C:/Users/L/.convokit/downloads/"
corpus = Corpus(filename=path + 'conversations-gone-awry-cmv-corpus')

def load_dfs(corpus):
    speakers = corpus.get_speakers_dataframe().drop(columns=['vectors'])
    
    conversations = corpus.get_conversations_dataframe().drop(columns=['vectors'])
    utterances = corpus.get_utterances_dataframe().drop(columns=['vectors'])
    # print(type(speakers), type(conversations), type(utterances))
    return speakers, conversations, utterances

speakers, conversations, utterances = load_dfs(corpus)

In [15]:
import string

def is_word(token):
    return token

def count_words(data):
    word_count = 0
    for item in data:
        for tok_info in item['toks']:
            token = tok_info['tok']
            if is_word(token):
                word_count += 1
    return word_count

print(utterances.head(n=1)['meta.parsed'].values[0][1]['toks'])
print(count_words(utterances.head(n=1)['meta.parsed'].values[0]))

[{'tok': 'and', 'tag': 'CC', 'dep': 'cc', 'up': 4, 'dn': []}, {'tok': 'I', 'tag': 'PRP', 'dep': 'nsubj', 'up': 4, 'dn': []}, {'tok': "'ve", 'tag': 'VB', 'dep': 'aux', 'up': 4, 'dn': []}, {'tok': 'always', 'tag': 'RB', 'dep': 'advmod', 'up': 4, 'dn': []}, {'tok': 'been', 'tag': 'VBN', 'dep': 'ROOT', 'dn': [0, 1, 2, 3, 5, 6, 14]}, {'tok': 'unsuccessful', 'tag': 'JJ', 'dep': 'acomp', 'up': 4, 'dn': []}, {'tok': 'in', 'tag': 'IN', 'dep': 'prep', 'up': 4, 'dn': [7]}, {'tok': 'convincing', 'tag': 'JJ', 'dep': 'pcomp', 'up': 6, 'dn': [8, 9]}, {'tok': 'people', 'tag': 'NNS', 'dep': 'dobj', 'up': 7, 'dn': []}, {'tok': 'about', 'tag': 'IN', 'dep': 'prep', 'up': 7, 'dn': [12]}, {'tok': 'why', 'tag': 'WRB', 'dep': 'advmod', 'up': 12, 'dn': []}, {'tok': 'they', 'tag': 'PRP', 'dep': 'nsubj', 'up': 12, 'dn': []}, {'tok': "'re", 'tag': 'VBP', 'dep': 'pcomp', 'up': 9, 'dn': [10, 11, 13]}, {'tok': 'wrong', 'tag': 'JJ', 'dep': 'acomp', 'up': 12, 'dn': []}, {'tok': '.', 'tag': '.', 'dep': 'punct', 'up': 4

In [33]:
# iterate over the list of dataframes, for each dataframe, iterate it rows' index and get the corresponding utterance id, then get the corresponding [meta.score] from the utterances dataframe

for df in df_list:
    df['meta.score'] = df.index.map(lambda idx: utterances.loc[idx, 'meta.score'])
    df['meta.score'] = df['meta.score'].astype(int)
    df_reset = df.reset_index()
    df['conv_pos'] = ((df_reset.index+1) / len(df_reset)).values
    df['token_count'] = df.index.map(lambda idx: count_words(utterances.loc[idx, 'meta.parsed']))

# 输出结果
for df in df_list:
    print(df)
        
with open ('before_random_forest.pkl', 'wb') as f:
    pickle.dump(df_list, f)
    
        # create a new column in the dataframe and assign the value of the score to it
        

                                                      text  \
id                                                           
cvs92ym  &gt;So within our predefined two genders, we h...   
cvstt70  &gt; But saying you're a boy because you act m...   
cvsvr2g  Because I have a penis, and balls, and my chro...   
cvsvwuy  &gt; Because I have a penis, and balls, and my...   
cvsxaph  Like I said a million times before. Exceptions...   

                                                     agu_1  \
id                                                           
cvs92ym  {"summarization": "The speaker expresses frust...   
cvstt70  {"summarization": "The speaker expresses stron...   
cvsvr2g  I cannot summarize the Reddit speaker's emotio...   
cvsvwuy  {"summarization": "The speaker expresses frust...   
cvsxaph  {"summarization": "The speaker is likely frust...   

                                                       imm  imm_check  \
id                                                       

In [22]:
# load the dataframe columns into the X and y variables numpy. y is imm_check (1 is True and 0 if False) and X is a vector that contains: values from ['txt_sentiment'] (which should be a list containing 4 floats), from ['txt_blob'] (which is a list contains 2 floats), from ['conv_pos'] (which is a float), from ['token_count'] (which is a int), from ['meta.score'] (which is a int)

# X = np.array([df[['txt_sentiment', 'txt_blob', 'conv_pos', 'token_count', 'meta.score']].values for df in df_list])
# y = np.array([df['imm_check'].values for df in df_list])

vectors = []

for df in df_list:
    for index, row in df.iterrows():
        # Extract and flatten the lists, then combine into a single vector
        vector = row['txt_sentiment'] + row['txt_blob'] + [row['conv_pos']] + [row['token_count']] + [row['meta.score']]
        vectors.append(vector)
        print(vector)
        print(1)

# Convert the list of vectors to a Numpy array
X = np.array(vectors)
Y = np.array([df['imm_check'].values[0] for df in df_list])

print(X)
print("\nY:")
print(Y)

[0.04, 0.092, 0.868, -0.6548, 0.009090909090909115, 0.43787878787878787, 0.2, 139, 1]
1
[0.036, 0.046, 0.918, -0.2263, 0.05, 0.18645833333333334, 0.4, 102, 1]
1
[0.0, 0.0, 1.0, 0.0, 0.0, 0.1, 0.6, 25, 1]
1
[0.027, 0.129, 0.844, -0.8451, -0.135, 0.44571428571428573, 0.8, 135, 1]
1
[0.048, 0.05, 0.902, -0.0433, -0.05113636363636364, 0.4231060606060606, 1.0, 134, 1]
1
[0.042, 0.147, 0.811, -0.9922, -0.03289473684210526, 0.47185672514619875, 0.125, 404, 1]
1
[0.074, 0.173, 0.752, -0.9879, -0.08337053571428572, 0.47976190476190483, 0.25, 317, -2]
1
[0.037, 0.105, 0.857, -0.8895, 0.005918560606060618, 0.5588541666666668, 0.375, 169, 3]
1
[0.115, 0.11, 0.775, -0.144, 0.05787878787878789, 0.47208333333333335, 0.5, 81, 0]
1
[0.049, 0.059, 0.891, -0.1921, 0.137797619047619, 0.22946428571428573, 0.625, 89, 0]
1
[0.066, 0.054, 0.88, 0.177, -0.056481481481481466, 0.33055555555555555, 0.75, 71, -1]
1
[0.081, 0.074, 0.844, -0.5405, -0.00897619047619047, 0.5106190476190475, 0.875, 369, 0]
1
[0.04, 0.0

In [32]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# 示例 DataFrame 列表
# df_list = [
#     pd.DataFrame({
#         'txt_sentiment': [[0.04, 0.092, 0.868, 0.96]],
#         'txt_blob': [[0.2, 0.3]],
#         'conv_pos': [0.2],
#         'token_count': [139],
#         'meta.score': [1],
#         'imm_check': [True]  # 目标变量
#     }),
#     pd.DataFrame({
#         'txt_sentiment': [[0.036, 0.046, 0.918, 0.92]],
#         'txt_blob': [[0.4, 0.5]],
#         'conv_pos': [0.4],
#         'token_count': [102],
#         'meta.score': [1],
#         'imm_check': [False]  # 目标变量
#     })
#     # 添加更多 DataFrames 如有需要
# ]

# 提取和组合值到向量用于 X
vectors = []
targets = []

for df in df_list:
    for index, row in df.iterrows():
        # 提取和展开列表，然后组合成一个向量
        vector = row['txt_sentiment'] + row['txt_blob'] + [row['conv_pos']] + [row['token_count']] + [row['meta.score']]
        vectors.append(vector)
        targets.append(row['imm_check'])

# 将向量列表转换为 Numpy 数组用于 X
X = np.array(vectors)

# 将目标值列表转换为 Numpy 数组用于 Y
Y = np.array(targets)

# 检查 X 和 Y 的长度是否一致
print(f"Length of X: {len(X)}")
print(f"Length of Y: {len(Y)}")

# 确保长度相同后继续
if len(X) == len(Y):
    # 创建随机森林分类器
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    
    # 使用交叉验证进行评估
    scores = cross_val_score(rf, X, Y, cv=5, scoring='accuracy')
    
    # 训练随机森林模型
    rf.fit(X, Y)
    
    # 打印交叉验证的评分结果
    print(f"Cross-validation scores: {scores}")
    print(f"Mean cross-validation score: {scores.mean()}")
else:
    print("Error: X and Y have different lengths. Please check the data extraction process.")


Length of X: 3877
Length of Y: 3877
Cross-validation scores: [0.75515464 0.76546392 0.75612903 0.74322581 0.73419355]
Mean cross-validation score: 0.750833388759561


In [17]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pickle


# 定义随机森林的超参数网格
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 创建随机森林分类器
rf = RandomForestClassifier(random_state=42)

# 使用 GridSearchCV 进行超参数搜索
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# 进行超参数搜索
grid_search.fit(X, Y)

# 保存计算结果到文件
with open('grid_search_results.pkl', 'wb') as f:
    pickle.dump(grid_search, f)

# 加载计算结果
with open('grid_search_results.pkl', 'rb') as f:
    loaded_grid_search = pickle.load(f)

# 打印最佳结果
print(f"Best parameters found: {loaded_grid_search.best_params_}")
print(f"Best cross-validation score: {loaded_grid_search.best_score_}")


Best parameters found: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}
Best cross-validation score: 0.7397439308280679


In [36]:
with open ('before_random_forest.pkl', 'rb') as f:
    df_list = pickle.load(f)
    
vectors = []
targets = []

for df in df_list:
    for index, row in df.iterrows():
        # 提取和展开列表，然后组合成一个向量
        vector = row['txt_sentiment'] + row['txt_blob'] + [row['conv_pos']] + [row['token_count']] + [row['meta.score']]
        vectors.append(vector)
        targets.append(row['imm_check'])

# 将向量列表转换为 Numpy 数组用于 X
X = np.array(vectors)

# 将目标值列表转换为 Numpy 数组用于 Y
Y = np.array(targets)

In [12]:
import numpy as np

column_labels = [
    'pos', 'neg', 'neu', 'compound',
    'polarity', 'subjectivity',
    'conv_pos',
    'token_count',
    'score'
]

def remove_columns(X, column_labels, columns_to_remove):
    indices_to_keep = [i for i, label in enumerate(column_labels) if label not in columns_to_remove]    
    X_new = X[:, indices_to_keep]
    return X_new

columns_to_remove = ['neg', 'compound', 'token_count']

X_new = remove_columns(X, column_labels, columns_to_remove)

print("Original X:")
print(X)
print("\nNew X after removing columns:")
print(X_new)


Original X:
[[4.00e-02 9.20e-02 8.68e-01 ... 2.00e-01 1.39e+02 1.00e+00]
 [3.60e-02 4.60e-02 9.18e-01 ... 4.00e-01 1.02e+02 1.00e+00]
 [0.00e+00 0.00e+00 1.00e+00 ... 6.00e-01 2.50e+01 1.00e+00]
 ...
 [8.70e-02 1.80e-01 7.33e-01 ... 5.00e-01 2.90e+01 5.40e+01]
 [0.00e+00 0.00e+00 1.00e+00 ... 7.50e-01 3.80e+01 5.00e+00]
 [0.00e+00 0.00e+00 1.00e+00 ... 1.00e+00 3.80e+01 3.40e+01]]

New X after removing columns:
[[ 4.00000000e-02  8.68000000e-01  9.09090909e-03  4.37878788e-01
   2.00000000e-01  1.00000000e+00]
 [ 3.60000000e-02  9.18000000e-01  5.00000000e-02  1.86458333e-01
   4.00000000e-01  1.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  1.00000000e-01
   6.00000000e-01  1.00000000e+00]
 ...
 [ 8.70000000e-02  7.33000000e-01  2.18750000e-01  6.12500000e-01
   5.00000000e-01  5.40000000e+01]
 [ 0.00000000e+00  1.00000000e+00  6.25000000e-02  2.75000000e-01
   7.50000000e-01  5.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00 -7.00000000e-02  4.30000000e-01
   1.0000

In [46]:
columns_to_remove_list = [['neg', 'pos', 'compound', 'score'], []]
X_new_list = [remove_columns(X, column_labels, columns_to_remove) for columns_to_remove in columns_to_remove_list]
grid = {
    'n_estimators': list(range(10, 110, 20)),
    'max_depth': list(range(3, 15, 3)),
    'min_samples_split': list(range(2, 11, 2)),
    'min_samples_leaf': [1, 2, 3, 4]
}
for X_new in X_new_list:
    print(auto_search(X_new, Y, grid=grid)[1:])

C:\Users\L\anaconda3\envs\Raja\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


({'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 6, 'n_estimators': 70}, 0.7645071499833722)
({'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 50}, 0.7639913535084802)


In [1]:
import pickle
# load the dataframe from combat_df_list_imms_1.pkl
with open ('before_random_forest.pkl', 'rb') as f:
    df_list = pickle.load(f)
    
with open( 'combat_df_list_imms_1.pkl', 'rb') as f:
    df_list_imms = pickle.load(f)
    
ls = []
ls_imms = []

for df in df_list:
    ls.append(df)

for sublist in df_list_imms:
    for df in sublist:
        ls_imms.append(df)
        

In [9]:
# Add the 'imm_1_check' column from df_list_imms to df_list
for df, df_imm in zip(ls, ls_imms):
    df['imm_1_check'] = df_imm['imm_1_check']
    
print(df_list[0])
# Save the updated dataframes to a new pickle file
with open('updated_before_random_forest.pkl', 'wb') as f:
    pickle.dump(df_list, f)
    

                                                      text  \
id                                                           
cvs92ym  &gt;So within our predefined two genders, we h...   
cvstt70  &gt; But saying you're a boy because you act m...   
cvsvr2g  Because I have a penis, and balls, and my chro...   
cvsvwuy  &gt; Because I have a penis, and balls, and my...   
cvsxaph  Like I said a million times before. Exceptions...   

                                                     agu_1  \
id                                                           
cvs92ym  {"summarization": "The speaker expresses frust...   
cvstt70  {"summarization": "The speaker expresses stron...   
cvsvr2g  I cannot summarize the Reddit speaker's emotio...   
cvsvwuy  {"summarization": "The speaker expresses frust...   
cvsxaph  {"summarization": "The speaker is likely frust...   

                                                       imm  imm_check  \
id                                                       

In [38]:
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, roc_auc_score

scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, average='macro'),
    'recall': make_scorer(recall_score, average='macro'),
    'f1': make_scorer(f1_score, average='macro'),
    'roc_auc': make_scorer(roc_auc_score, average='macro', multi_class='ovr')
}

def auto_search(X, Y, grid=param_grid):
    rf = RandomForestClassifier(random_state=42)
    grid_search = GridSearchCV(estimator=rf, param_grid=grid, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X, Y)
    
    # 提取所有评分指标的结果
    cv_results = grid_search.cv_results_
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    
    # 提取最佳参数的所有评分指标
    best_index = grid_search.best_index_
    best_scores = {metric: cv_results[f'mean_test_{metric}'][best_index] for metric in scoring.keys() if f'mean_test_{metric}' in cv_results}
    return grid_search, grid_search.best_params_, grid_search.best_score_,best_scores

In [42]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pickle
# 提取和组合值到向量用于 X

with open('updated_before_random_forest.pkl', 'rb') as f:
    df_list = pickle.load(f)
vectors = []
targets = []

for df in df_list:
    for index, row in df.iterrows():
        # print(1)
        # 提取和展开列表，然后组合成一个向量
        vector = row['txt_sentiment'] + row['txt_blob'] + [row['conv_pos']] + [row['token_count']] + [row['meta.score']]
        vectors.append(vector)
        # 计算imms_check和imms_check_1的平均值
        x = (int(row['imm_check']) + int(row['imm_1_check']))
        if x==0:
            a=1
        else:
            a=0
        targets.append(a)

X = np.array(vectors)

Y = np.array(targets)

rf = RandomForestClassifier(random_state=42)
rf.fit(X, Y)

# 获取特征重要性
feature_importances = rf.feature_importances_


column_labels = [
    'pos', 'neg', 'neu', 'compound',
    'polarity', 'subjectivity',
    'conv_pos', 'token_count', 'score'
]

# 打印特征重要性
feature_importance_df = pd.DataFrame({
    'feature': column_labels,
    'importance': feature_importances
}).sort_values(by='importance', ascending=False)

print(feature_importance_df)

        feature  importance
3      compound    0.127009
4      polarity    0.125157
5  subjectivity    0.124406
7   token_count    0.120875
2           neu    0.120235
1           neg    0.107143
6      conv_pos    0.105160
0           pos    0.093956
8         score    0.076059


In [3]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 假设X和Y是你的数据集
# X = ...  # 你的特征数据
# Y = ...  # 你的标签数据

# 定义多个评分指标
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, average='macro'),
    'recall': make_scorer(recall_score, average='macro'),
    'f1': make_scorer(f1_score, average='macro'),
    'roc_auc': make_scorer(roc_auc_score, average='macro', multi_class='ovr')
}

# 定义参数网格
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 定义函数进行自动搜索和评估
def auto_search_gbdt(X, Y, grid=param_grid):
    gbdt = GradientBoostingClassifier(random_state=42)
    grid_search = GridSearchCV(estimator=gbdt, param_grid=grid, cv=5, scoring=scoring, refit='f1', n_jobs=-1)
    grid_search.fit(X, Y)
    
    # 提取所有评分指标的结果
    cv_results = grid_search.cv_results_
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    
    # 提取最佳参数的所有评分指标
    best_index = grid_search.best_index_
    best_scores = {metric: cv_results[f'mean_test_{metric}'][best_index] for metric in scoring.keys() if f'mean_test_{metric}' in cv_results}
    
    return grid_search, best_params, best_score, best_scores

# 进行自动搜索和评估
grid_search, best_params, best_score, best_scores = auto_search_gbdt(X, Y, grid=param_grid)

# 输出最佳参数和评分
print(f'Best Params: {best_params}')
print(f'Best Accuracy: {best_score}')
print(f'Best Scores: {best_scores}')


NameError: name 'X' is not defined

In [28]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import itertools
import pickle

# Define the auto_search function
def auto_search(X, Y, grid):
    rf = RandomForestClassifier(random_state=42)
    grid_search = GridSearchCV(estimator=rf, param_grid=grid, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X, Y)
    return grid_search, grid_search.best_params_, grid_search.best_score_

# Define the remove_columns function
def remove_columns(X, column_labels, columns_to_remove):
    indices_to_keep = [i for i, label in enumerate(column_labels) if label not in columns_to_remove]
    X_new = X[:, indices_to_keep]
    return X_new

# Define the column labels
column_labels = [
    'pos', 'neg', 'neu', 'compound',
    'polarity', 'subjectivity',
    'conv_pos', 'token_count', 'score'
]

# Define the grid for hyperparameter search
grid = {
    'n_estimators': list(range(10, 110, 20)),
    'max_depth': [None] + list(range(1, 15, 3)),
    'min_samples_split': list(range(2, 11, 2)),
    'min_samples_leaf': [1, 2, 3, 4]
}

# Load the data
with open('updated_before_random_forest.pkl', 'rb') as f:
    df_list = pickle.load(f)

# Extract vectors and targets
vectors = []
targets = []

for df in df_list:
    for index, row in df.iterrows():
        vector = row['txt_sentiment'] + row['txt_blob'] + [row['conv_pos']] + [row['token_count']] + [row['meta.score']]
        vectors.append(vector)
        x = (int(row['imm_check']) + int(row['imm_1_check']))
        if x == 0:
            a = 1
        else: a=0
        targets.append(a)

X = np.array(vectors)
Y = np.array(targets)

# Generate all possible combinations of columns to remove
all_combinations = []
for r in range(len(column_labels) + 1):
    combinations_object = itertools.combinations(column_labels, r)
    combinations_list = list(combinations_object)
    all_combinations.extend(combinations_list)

# Iterate over each combination, perform grid search, and save results
results = []

for columns_to_remove in all_combinations:
    X_new = remove_columns(X, column_labels, columns_to_remove)
    _, best_params, best_score = auto_search(X_new, Y, grid)
    print(columns_to_remove)
    print(best_score)
    results.append((columns_to_remove, best_params, best_score))

# Print the results
for columns_to_remove, best_params, best_score in results:
    print(f"Columns removed: {columns_to_remove}")
    print(f"Best parameters: {best_params}")
    print(f"Best score: {best_score}\n")

()
0.95228300631859
('pos',)
0.95228300631859
('neg',)
0.9520249418024609
('neu',)
0.95228300631859
('compound',)
0.9522826737612238


C:\Users\L\anaconda3\envs\Raja\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


('polarity',)
0.95228300631859


C:\Users\L\anaconda3\envs\Raja\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


('subjectivity',)
0.95228300631859


...

C:\Users\L\anaconda3\envs\Raja\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


('neu', 'subjectivity')
0.9520249418024609
('neu', 'conv_pos')
0.9527988027934817
('neu', 'token_count')
0.9522826737612238
('neu', 'score')
0.9520249418024609
('compound', 'polarity')
0.9522826737612238
('compound', 'subjectivity')
0.9522826737612238


C:\Users\L\anaconda3\envs\Raja\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


('compound', 'conv_pos')
0.9525407382773527
('compound', 'token_count')
0.95228300631859
('compound', 'score')
0.95228300631859


KeyboardInterrupt: 